基于坐标轴数据集的多标签分类

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, random_split

In [2]:
directions = [
    (1, 1),
    (-1, 1),
    (-1, -1), 
    (1, -1)
]
directions = torch.tensor(directions, dtype=torch.float32)

In [3]:
data = torch.randn(1000, 2).abs()
data = data.unsqueeze(1)

In [4]:
data.shape, directions.shape

(torch.Size([1000, 1, 2]), torch.Size([4, 2]))

In [5]:
dataset_data = data * directions  # (1000, 4, 2)
dataset_data = dataset_data.reshape(-1, 2)
print(dataset_data.shape)

torch.Size([4000, 2])


In [6]:
labels = torch.where(dataset_data > 0, torch.tensor(1.), torch.tensor(0.))
labels = labels.reshape(-1, 2)
labels.shape

torch.Size([4000, 2])

In [7]:
dataset_data.shape, labels.shape

(torch.Size([4000, 2]), torch.Size([4000, 2]))

In [8]:
class MultiLabelDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return {
            "data": self.data[idx],
            "labels": self.labels[idx]
        }

In [9]:
full_dataset = MultiLabelDataset(dataset_data, labels)
train_dataset, eval_dataset = random_split(full_dataset, [0.8, 0.2])
len(train_dataset), len(eval_dataset)

(3200, 800)

In [10]:
train_dataset[0]

{'data': tensor([ 0.2244, -0.7083]), 'labels': tensor([1., 0.])}

In [11]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc = nn.Linear(2, 2, bias=False)
    
    def compute_loss(self, predictions, targets):
        # criterion = nn.BCELoss()
        criterion = nn.BCEWithLogitsLoss()
        loss = criterion(predictions, targets)
        return loss

    # def custom_sigmoid(self, x):
    #     return 1 / (2 + torch.exp(-x))

    def forward(self, data, labels=None):
        x = self.fc(data)
        return {
            "loss": self.compute_loss(x, labels) if labels is not None else None,
            "logits": x
        }

In [12]:
from transformers import Trainer, TrainingArguments

c:\Users\jiell\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# def compute_metrics(eval_pred):
#     from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
#     import numpy as np

#     logits, labels = eval_pred
#     probs = torch.sigmoid(torch.tensor(logits)).numpy()
#     preds = (probs >= 0.5).astype(int)

#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds, average='macro')
#     recall = recall_score(labels, preds, average='macro')
#     f1 = f1_score(labels, preds, average='macro')

#     return {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1,
#     }

import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    # 将 logits 转换为概率并得到预测结果
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = (probs >= 0.5).astype(int)
    
    # 计算整体准确率
    accuracy = accuracy_score(labels, preds)
    
    # 计算每个类别的 Precision, Recall, F1
    precision_per_class = precision_score(labels, preds, average=None, zero_division=0)
    recall_per_class = recall_score(labels, preds, average=None, zero_division=0)
    f1_per_class = f1_score(labels, preds, average=None, zero_division=0)
    
    # 计算宏观平均指标
    precision_macro = precision_score(labels, preds, average='macro', zero_division=0)
    recall_macro = recall_score(labels, preds, average='macro', zero_division=0)
    f1_macro = f1_score(labels, preds, average='macro', zero_division=0)
    
    # 构建结果字典
    metrics = {
        # 'accuracy': accuracy,
        # 'precision_macro': precision_macro,
        # 'recall_macro': recall_macro,
        'f1': f1_macro
    }
    
    # 添加每个类别的指标
    for i, (p, r, f) in enumerate(zip(precision_per_class, recall_per_class, f1_per_class)):
        metrics[f'f1_class_{i}'] = f
    
    return metrics

In [17]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="logs",
    report_to="tensorboard",
    remove_unused_columns=False,
    logging_steps=10,
    learning_rate=5e-3,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [18]:
trainer = Trainer(
    model=CustomModel(),
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,F1,F1 Class 0,F1 Class 1
1,0.561300,0.556013,0.783887,0.990220,0.577554
2,0.403300,0.391975,0.996807,0.998782,0.994832
3,0.305600,0.313782,0.997416,1.000000,0.994832
4,0.300500,0.269982,0.996199,0.997567,0.994832
5,0.244700,0.243305,0.996843,0.997567,0.996119
6,0.231900,0.226746,0.996199,0.997567,0.994832
7,0.207200,0.216733,0.997450,0.998782,0.996119
8,0.214000,0.211438,0.996199,0.997567,0.994832
9,0.215400,0.209313,0.997450,0.998782,0.996119
10,0.216800,0.208979,0.997450,0.998782,0.996119


TrainOutput(global_step=2000, training_loss=0.31414607852697374, metrics={'train_runtime': 3.7828, 'train_samples_per_second': 8459.453, 'train_steps_per_second': 528.716, 'total_flos': 0.0, 'train_loss': 0.31414607852697374, 'epoch': 10.0})

In [19]:
pred_data = torch.tensor([[0.5, 0.5], [0.1, 0.9], [0.01, 0.999], [-0.01, 0.999], [-99, 99]]).to("cuda")

In [20]:
logits = trainer.model(pred_data)["logits"]

In [21]:
nn.Sigmoid()(logits)

tensor([[0.8242, 0.7607],
        [0.5700, 0.8908],
        [0.4993, 0.9115],
        [0.4837, 0.9116],
        [0.0000, 1.0000]], device='cuda:0', grad_fn=<SigmoidBackward0>)